In [ ]:
from datetime import datetime as dt
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

def moving_average(x, w) -> np.ndarray:
    return np.convolve(x, np.ones(w), 'valid') / w

def load_csv(file):
    datetime = []
    temp = []
    humd = []
    power = []
    with open(file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            d, t, h, p = row
            datetime.append(dt.fromisoformat(d).timestamp())
            temp.append(float(t))
            humd.append(float(h))
            power.append(float(p))
    return (datetime, temp, humd, power)

def plot(x, y, title):
    data_copy = np.array(y)
    data_norm = (data_copy - data_copy.mean()) / data_copy.max()

    yf = fft(data_norm)
    xf = fftfreq(len(yf), 1 / 288)[:len(yf)//2]
    yf = np.abs(yf[:len(yf)//2])

    fig, axs = plt.subplots(1, 3, tight_layout=True)
    fig.set_figwidth(20)
    fig.suptitle(title)

    print(f"Mean: {np.mean(y).round(4)}")
    print(f"FFT (max): {xf[np.argmax(yf)]}")

    axs[0].set_title("Time series")
    axs[0].axis([0, 24, None, None])
    axs[0].axhline(y=np.mean(y), c='r', label='mean')
    axs[0].legend()
    axs[0].plot(x, y)
    axs[1].set_title("Histogram")
    axs[1].axvline(x=np.mean(y), c='r', label='mean')
    axs[1].legend()
    axs[1].hist(y)
    axs[2].set_title("FFT")
    axs[2].axvline(x=xf[np.argmax(yf)], c='r', label='max')
    axs[2].legend()
    axs[2].plot(xf, yf / yf.max())

Find label index

In [ ]:
time_table = {
    '2023-05-29': ['06:30:00', '18:00:00'],
    '2023-05-30': ['07:00:00', '19:00:00'],
    '2023-05-31': ['06:00:00', '20:00:00'],
    '2023-06-01': ['07:30:00', '18:00:00'],
    '2023-06-02': ['08:00:00', '19:00:00'],
    '2023-06-03': ['06:00:00', '21:00:00'],
    '2023-06-04': ['07:30:00', '22:00:00'],
    '2023-06-05': ['05:00:00', '22:00:00'],
    '2023-06-06': ['06:00:00', '21:00:00'],
}

In [ ]:
# from datetime import datetime as dt
# import numpy as np

# morning_datetime = dt.fromisoformat(f'{file} {time_table[file][0]}')
# evening_datetime = dt.fromisoformat(f'{file} {time_table[file][1]}')

# morning_diff_lst = []
# evening_diff_lst = []

# for item in datetime_data:
#     d = dt.fromisoformat(item)
#     m_diff = abs(d.timestamp() - morning_datetime.timestamp())
#     e_diff = abs(d.timestamp() - evening_datetime.timestamp())
#     morning_diff_lst.append(m_diff)
#     evening_diff_lst.append(e_diff)

# morning_index = np.argmin(morning_diff_lst)
# evening_index = np.argmin(evening_diff_lst)
# print(time_table[file])
# datetime_data[morning_index].split(' ')[1], datetime_data[evening_index].split(' ')[1]

Load

In [ ]:
file = '2023-07-05'
datetime_data, temp_data, humd_data, power_data = load_csv(f"zuraach_ail2/{file}_raw.csv")

In [ ]:
temp_x = [float(dt.fromtimestamp(item).hour) + float(dt.fromtimestamp(item).minute)/60 for item in datetime_data]
plot(temp_x, temp_data, "RAW TEMPERATURE DATA")
plot(temp_x, humd_data, "RAW HUMIDITY DATA")
plot(temp_x, power_data, "POWER")

In [ ]:
td = np.array(temp_data)
hd = np.array(humd_data)
nd = np.array(power_data)
plt.plot(temp_x, td / td.max(), label='temp')
plt.plot(temp_x, hd / hd.max(), label='humd')
plt.plot(temp_x, nd / nd.max(), label='power')
plt.legend()

Temperature data filter

In [ ]:
WINDOW = 20

temp_filtered = moving_average(temp_data, WINDOW)
datetime_filtered = moving_average(datetime_data, WINDOW).tolist()

temp_x = [float(dt.fromtimestamp(item).hour) + float(dt.fromtimestamp(item).minute)/60 for item in datetime_filtered]
plot(temp_x, temp_filtered, "Temp filtered")

Rate of change calculation

In [ ]:
rate_of_change = []
roc_datetime = []

for i in range(len(datetime_filtered) - 1):
    current_date = datetime_filtered[i]
    next_date = datetime_filtered[i + 1]
    current_temp = temp_filtered[i]
    next_temp = temp_filtered[i + 1]

    dtime = (next_date - current_date) / 60  # mins
    dtemp = next_temp - current_temp
    rate_of_change.append(dtemp / dtime)
    roc_datetime.append((next_date + current_date) / 2)

roc_x = [float(dt.fromtimestamp(item).hour) + float(dt.fromtimestamp(item).minute)/60 for item in roc_datetime]
plot(roc_x, rate_of_change, "Rate of change")